In [ ]:

from basicpy import BaSiC

from skimage.io import imread

import numpy as np

import napari
from bioio import BioImage
import bioio_nd2

from napari_animation import Animation
from napari.utils.colormaps import label_colormap

In [ ]:
labels_1ch = imread("stardist_labels_1_channel.tif")
labels_2ch = imread("stardist_labels_2_channel.tif")
labels_3ch = imread("stardist_labels_3_channel.tif")

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.window.resize(1200, 800)
viewer.reset_view()

In [ ]:
image = BioImage("../2.nd2", reader=bioio_nd2.Reader)
scale = (image.physical_pixel_sizes.Y, image.physical_pixel_sizes.X)
cyan = image.get_image_data("TYX", C=3)
magenta = image.get_image_data("TYX", C=0)
tubulin = image.get_image_data("TYX", C=1)

# flat field correction
basic_cyan = BaSiC.load_model("basic_model_cyan")
basic_magenta = BaSiC.load_model("basic_model_magenta")
basic_tubulin = BaSiC.load_model("basic_model_tubulin")

cyan = basic_cyan.transform(cyan, timelapse=True)
magenta = basic_magenta.transform(magenta, timelapse=True)
tubulin = basic_tubulin.transform(tubulin, timelapse=True)

mag_1_perc = np.percentile(magenta, 1)
mag_99_9_perc = np.percentile(magenta, 99.9)
cyan_1_perc = np.percentile(cyan, 1)
cyan_99_9_perc = np.percentile(cyan, 99.9)
tub_1_perc = np.percentile(tubulin, 1)
tub_99_9_perc = np.percentile(tubulin, 99.9)

In [ ]:
viewer.scale_bar.unit = "um"
viewer.scale_bar.font_size = 0
viewer.scale_bar.ticks = False

In [ ]:
old_time = -1
def update_slider(event):
    time = viewer.dims.current_step[0]
    global old_time
    if time != old_time:
        old_time = time
    else:
        return
    time = 15.0 * time
    viewer.text_overlay.text = f"{round(time)} min "

In [ ]:
viewer.text_overlay.color = "white"
viewer.text_overlay.blending = "translucent_no_depth"
viewer.text_overlay.position = "top_left"
viewer.text_overlay.font_size = 18
viewer.text_overlay.visible = True
viewer.dims.events.current_step.connect(update_slider)

In [ ]:
viewer.scale_bar.font_size = 0
viewer.scale_bar.unit = "um"
viewer.scale_bar.font_size = 0
viewer.scale_bar.visible = True

In [ ]:
viewer.dims.current_step = (0, 0, 0)

In [ ]:
crop_range_y = slice(600, 2200)
crop_range_x = slice(400, 2000)

In [ ]:
cyan_layer = viewer.add_image(cyan[:, crop_range_y, crop_range_x], colormap="cyan", blending="additive", scale=scale)
magenta_layer = viewer.add_image(magenta[:, crop_range_y, crop_range_x], colormap="magenta", blending="additive", scale=scale)
tubulin_layer = viewer.add_image(tubulin[:, crop_range_y, crop_range_x], colormap="gray", blending="additive", scale=scale)

In [ ]:
# this did not work because of this issue: https://github.com/napari/napari-animation/issues/226
#lab_1ch_layer = viewer.add_labels(labels_1ch[:, crop_range_y, crop_range_x], colormap = DirectLabelColormap(color_dict={None: "yellow"}))
#lab_2ch_layer = viewer.add_labels(labels_2ch[:, crop_range_y, crop_range_x], colormap = DirectLabelColormap(color_dict={None: "yellow"}))
#lab_3ch_layer = viewer.add_labels(labels_3ch[:, crop_range_y, crop_range_x], colormap = DirectLabelColormap(color_dict={None: "yellow"}))

In [ ]:
colormap = label_colormap(num_colors=6)

labels_1ch[labels_1ch > 0] = 5
labels_2ch[labels_2ch > 0] = 5
labels_3ch[labels_3ch > 0] = 5

lab_1ch_layer = viewer.add_labels(labels_1ch[:, crop_range_y, crop_range_x], colormap = colormap, scale=scale)
lab_2ch_layer = viewer.add_labels(labels_2ch[:, crop_range_y, crop_range_x], colormap = colormap, scale=scale)
lab_3ch_layer = viewer.add_labels(labels_3ch[:, crop_range_y, crop_range_x], colormap = colormap, scale=scale)

In [ ]:
lab_1ch_layer.contour = 14
lab_2ch_layer.contour = 14
lab_3ch_layer.contour = 14

In [ ]:
viewer.dims.current_step = (0, 0, 0)

magenta_layer.contrast_limits = (mag_1_perc, mag_99_9_perc)
cyan_layer.contrast_limits = (cyan_1_perc, cyan_99_9_perc)
tubulin_layer.contrast_limits = (tub_1_perc, tub_99_9_perc)

In [ ]:
animation = Animation(viewer)

tubulin_layer.visible = True
cyan_layer.visible = False
magenta_layer.visible = False

lab_1ch_layer.visible = True
lab_2ch_layer.visible = False
lab_3ch_layer.visible = False

# start animation on first frame
viewer.dims.current_step = (0, 0, 0)
animation.capture_keyframe()
# last frame
viewer.dims.current_step = (labels_1ch.shape[0] - 1, 0, 0)
animation.capture_keyframe(steps=labels_1ch.shape[0] - 1)
animation.animate("movie_1ch_cropped.mp4",
                   canvas_only=True, fps=4, quality=9, scale_factor=1.0)

In [ ]:
animation2 = Animation(viewer)

tubulin_layer.visible = False
cyan_layer.visible = True
magenta_layer.visible = True

lab_2ch_layer.visible = True
lab_1ch_layer.visible = False

magenta_layer.contrast_limits = (mag_1_perc, mag_99_9_perc)
cyan_layer.contrast_limits = (cyan_1_perc, cyan_99_9_perc)
tubulin_layer.contrast_limits = (tub_1_perc, tub_99_9_perc)

# start animation on first frame
viewer.dims.current_step = (0, 0, 0)
animation2.capture_keyframe()
# last frame
viewer.dims.current_step = (labels_1ch.shape[0] - 1, 0, 0)
animation2.capture_keyframe(steps=labels_1ch.shape[0] - 1)
animation2.animate("movie_2ch_cropped.mp4",
                   canvas_only=True, fps=4, quality=9, scale_factor=1.0)

In [ ]:
animation3 = Animation(viewer)

tubulin_layer.visible = True
cyan_layer.visible = True
magenta_layer.visible = True

lab_2ch_layer.visible = False
lab_3ch_layer.visible = True

magenta_layer.contrast_limits = (mag_1_perc, mag_99_9_perc)
cyan_layer.contrast_limits = (cyan_1_perc, cyan_99_9_perc)
tubulin_layer.contrast_limits = (tub_1_perc, tub_99_9_perc)

# start animation on first frame
viewer.dims.current_step = (0, 0, 0)
animation3.capture_keyframe()
# last frame
viewer.dims.current_step = (labels_1ch.shape[0] - 1, 0, 0)
animation3.capture_keyframe(steps=labels_1ch.shape[0] - 1)
animation3.animate("movie_3ch_cropped.mp4",
                   canvas_only=True, fps=4, quality=9, scale_factor=1.0)

In [ ]:
viewer.scale_bar.font_size = 10